<center>
    <h3> Movie Recommondation Systems </h3>
</center>

In [635]:
# import pakages 
import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [636]:
# Load the datasets
rating_df = pd.read_csv('u.data',sep = '\t', header = None)
rating_df.columns = ['user_id','movie_id','rating','timestamp']
rating_df.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [637]:
movies_df = pd.read_csv('u.item', sep = '|', encoding = 'latin-1', header = None)

In [638]:
movies_df = movies_df[[0,1]]
movies_df.columns = ['movie_id', 'movie_name']
movies_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [625]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100000 non-null  int64
 1   movie_id   100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [566]:
print('Total Ratings:', rating_df.rating.nunique())
print('Total Movies:', rating_df.movie_id.nunique())
print('Total Users:', rating_df.user_id.nunique())

Total Ratings: 5
Total Movies: 1682
Total Users: 943


In [567]:
# rating_given by users in stars
rating_df.rating.value_counts()

rating
4    34174
3    27145
5    21201
2    11370
1     6110
Name: count, dtype: int64

In [568]:
# no. of times ratings given by each user
rating_df.user_id.value_counts().head()

user_id
405    737
655    685
13     636
450    540
276    518
Name: count, dtype: int64

In [569]:
users_list = list(set(rating_df.user_id))
movies_list = list(set(rating_df.movie_id))

In [570]:
rating_df['Date'] = pd.to_datetime(rating_df['timestamp'], unit = 's')
rating_df.head(2)

,user_id,movie_id,rating,timestamp,Date
0,196,242,3,881250949,1997-12-04 15:55:49
1,186,302,3,891717742,1998-04-04 19:22:22


In [571]:
# total years
rating_df.Date.dt.year.unique()

array([1997, 1998])

In [572]:
# Note: we can select any sepcific time period data because remondations are usually change with time so

In [573]:
rating_df = rating_df[["user_id", "movie_id", "rating"]]
rating_df

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [574]:
# no.of movies watched by each user
rating_df.groupby(['user_id'])[['movie_id']].nunique()

,movie_id
user_id,
1,272
2,62
3,54
4,24
5,175
...,...
939,49
940,107
941,22


In [575]:
user_movie_rating = pd.pivot_table(data = rating_df, index = 'user_id', columns= 'movie_id', values= 'rating', aggfunc= 'mean', fill_value= 0)
user_movie_rating

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### User Based Similarity

In [576]:
# finding users similarities
user_user_similarity = pairwise_distances(user_movie_rating, metric='euclidean')
user_user_similarity

array([[ 0.        , 65.2533524 , 65.91661399, ..., 63.17436189,
        67.74215822, 61.8950725 ],
       [65.2533524 ,  0.        , 35.62302626, ..., 33.24154028,
        44.71017781, 53.24471805],
       [65.91661399, 35.62302626,  0.        , ..., 28.10693865,
        41.88078318, 51.6139516 ],
       ...,
       [63.17436189, 33.24154028, 28.10693865, ...,  0.        ,
        41.32795664, 49.2138192 ],
       [67.74215822, 44.71017781, 41.88078318, ..., 41.32795664,
         0.        , 55.13619501],
       [61.8950725 , 53.24471805, 51.6139516 , ..., 49.2138192 ,
        55.13619501,  0.        ]])

In [577]:
# here each row is one user similarity vector , as lower as similar and as higher as dissimiler because we are using metrics == 'euclidean'
user_user_similarity = pd.DataFrame(user_user_similarity, columns = users_list, index = users_list)
user_user_similarity

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
1,0.000000,65.253352,65.916614,65.360539,61.147363,63.348244,78.689262,60.695964,64.776539,67.527772,66.430415,61.057350,81.178815,63.031738,65.444633,63.780875,61.684682,66.309879,63.395583,61.684682,67.601775,60.868711,59.253692,63.150614,61.286214,63.047601,63.126856,61.967734,64.031242,63.592452,64.907627,62.289646,65.107603,65.368188,64.443774,65.099923,61.546730,69.713700,64.629715,64.117080,...,65.559134,64.039051,64.498062,73.878278,62.153037,64.459289,61.627916,63.929649,63.718129,58.625933,64.163853,64.443774,58.060313,63.812225,64.876806,67.297845,64.342832,63.466527,60.282667,67.564784,62.128898,64.280635,63.639610,66.294796,62.481997,61.229078,61.032778,65.719099,71.874891,55.614746,64.591021,65.030762,67.111847,62.841069,66.279710,66.775744,62.369865,63.174362,67.742158,61.895073
2,65.253352,0.000000,35.623026,33.970576,50.009999,55.812185,84.699469,40.816663,33.734256,60.646517,53.563047,41.206796,84.439327,45.705580,32.726136,57.375953,31.288976,67.609171,32.908965,36.864617,44.877611,49.769469,51.390661,42.918527,43.611925,32.664966,31.796226,43.266615,34.117444,34.842503,37.188708,33.376639,32.695565,32.202484,33.060551,33.496268,39.924930,52.345009,30.870698,31.096624,...,35.721142,32.878564,33.376639,56.071383,41.194660,34.856850,32.372828,48.631266,41.182521,47.801674,33.689761,30.512293,65.749525,32.000000,45.705580,50.209561,29.461840,43.220366,46.882833,41.460825,40.174619,34.669872,28.106939,49.162994,38.574603,38.065733,33.331667,29.715316,68.963759,47.169906,54.626001,32.848135,45.166359,29.137605,39.153544,37.828561,42.355637,33.241540,44.710178,53.244718
3,65.916614,35.623026,0.000000,25.238859,47.222876,57.775427,83.958323,36.428011,29.410882,60.423505,51.633323,37.669616,84.350459,46.540305,36.851052,56.008928,27.748874,69.079664,26.267851,31.208973,42.367440,46.454279,50.695167,41.243181,41.844952,35.693137,27.676705,39.534795,28.792360,32.078030,29.664794,29.342802,22.803509,27.313001,23.958297,26.739484,36.041643,48.548944,24.494897,22.671568,...,33.481338,27.276363,32.326460,57.436922,36.823905,30.594117,33.421550,44.631827,35.707142,46.238512,27.495454,22.583180,64.930732,30.708305,42.071368,51.845926,21.424285,40.804412,44.463468,39.293765,40.062451,26.210685,23.216374,46.818800,33.689761,34.263683,33.286634,34.029399,67.253253,44.586994,54.543561,32.893768,47.822589,30.724583,40.224371,37.443290,40.236799,28.106939,41.880783,51.613952
4,65.360539,33.970576,25.238859,0.000000,46.701178,57.645468,83.282651,34.058773,28.284271,60.299254,50.803543,37.148351,86.011627,45.683695,36.152455,55.226805,26.888659,68.840395,24.959968,30.149627,41.617304,45.199558,49.789557,38.858718,40.595566,35.594943,26.400758,37.576588,28.495614,31.016125,31.320920,28.774989,23.173260,26.627054,24.145393,27.166155,33.852622,48.207883,24.919872,24.637370,...,32.893768,27.658633,32.465366,57.706152,36.013886,29.949958,31.176915,43.897608,35.327043,45.574115,26.925824,25.436195,64.536811,30.232433,41.821047,53.525695,25.139610,40.373258,42.614552,40.385641,39.471509,25.961510,23.452079,45.022217,33.555923,33.630343,31.984371,34.394767,66.543219,43.577517,53.870214,32.542280,48.249352,28.160256,39.484174,36.972963,39.242834,26.814175,40.853396,50.685304
5,61.147363,50.009999,47.222876,46.701178,0.000000,60.728906,77.272246,45.464272,45.727453,64.117080,52.839379,49.325450,81.553663,51.516987,50.517324,57.175169,44.045431,67.305275,43.289722,42.778499,52.602281,43.611925,48.083261,51.623638,45.552168,48.662100,44.000000,44.821870,46.378875,45.464272,47.644517,44.124823,45.891176,46.151923,44.766059,45.705580,44.710178,52.867760,45.825757,45.650849,...,48.548944,46.711883,47.465777,64.984614,46.195238,47.180504,44.977772,46.151923,46.529560,48.764741

In [578]:
# top 5 similar users to user 1 
user_user_similarity[1].sort_values( )[1:6].index   # NOte: here alwarys start with 1 not 0 in accessing because 0'th is for self comparing

Index([738, 521, 933, 715, 44], dtype='int64')

In [579]:
for i in [1,5,10,20,50]:

        # first find top similar users to user user i 
    top_2_similar_users = list(user_user_similarity[i].sort_values( )[1:3].index)
    print(f'Top 2 similar users to user {i} are : {top_2_similar_users}')
    
    # now find the movies which are watched by top similar users to recommond 
    for j in range(0,len(top_2_similar_users)):
        user_1 = i     # this is self user 
        user_2 = top_2_similar_users[j]        # this is his similar user 
        print(f'\n Movies to recommond to user {user_1} based on his similar user {user_2}')
        movies_user_1 = set(rating_df[rating_df['user_id']== user_1]['movie_id'].values)   # this is movies watched by self user 
        movies_user_2 = set(rating_df[rating_df['user_id']== user_2]['movie_id'].values)   # this is movies watched by his similar user
        rec_movies_list = list(movies_user_2 - movies_user_1)  # movies which are not watched by user 2 to recommond  
        print(list(movies_df[movies_df['movie_id'].isin(rec_movies_list)]['movie_name']),sep = ',')   # getting the movies names out of movie_df
    print('\n-------------------------------------------------------------------------------------------------------------------------------------------\n')

--------------------------------------------Recommondations for user: 1 -------------------------------------------------------------- 

Top 2 similar users to user 1 are : [738, 521]

 Movies to recommond to user 1 based on his similar user 738
['Face/Off (1997)', 'Titanic (1997)', "Schindler's List (1993)", 'Alien: Resurrection (1997)', "One Flew Over the Cuckoo's Nest (1975)", 'Clueless (1995)', 'Star Trek: Generations (1994)', 'True Lies (1994)', 'Mrs. Doubtfire (1993)', 'Batman (1989)', 'Mission: Impossible (1996)', 'Close Shave, A (1995)', 'Cinderella (1950)', 'E.T. the Extra-Terrestrial (1982)', 'Day the Earth Stood Still, The (1951)', 'Forbidden Planet (1956)', 'Star Trek: The Motion Picture (1979)', "Jackie Chan's First Strike (1996)", 'Tombstone (1993)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', "It's a Wonderful Life (1946)", 'Lawrence of Arabia (1962)', 'Manhattan (1979)', 'Gandhi (1982)', 'Killing Fields, The (1984)', 'Die Hard: With a 

### Item based similarity

In [556]:
# finding movies rating given by each user to give as input to pairwise similarity ot get similarities between movies 
movie_user_rating = rating_df.pivot_table(index = 'movie_id', columns = 'user_id', values= 'rating', aggfunc= 'mean', fill_value= 0)
movie_user_rating

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,1.0,5.0,4.0,5.0,0.0,3.0,5.0,0.0,5.0,0.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,4.0,3.0,3.0,4.0,0.0,3.0,5.0,3.0,5.0,0.0,0.0,5.0,0.0,3.0,3.0,0.0,4.0,3.0,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
2,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,5.0,5.0,0.0,0.0,5.0,0.0,3.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [592]:
# similarity between movies based on euclidean distance,  As low the distance as more similar and as high distance as discimilar movies
# here similarities between movies are given in row level
movie_movie_similarity = pairwise_distances(movie_user_rating, metric= 'euclidean')
movie_movie_similarity = pd.DataFrame(movie_movie_similarity, columns = movies_list, index = movies_list)
movie_movie_similarity

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,0.000000,77.723870,80.074965,76.896034,81.565924,84.734881,71.007042,77.006493,78.625696,82.752643,77.967942,82.534841,78.128100,83.588277,71.721684,82.643814,82.000000,84.882271,85.076436,83.180527,79.768415,77.562878,84.587233,75.332596,71.049279,83.312664,83.222593,70.213959,79.037966,84.622692,76.694198,82.969874,81.926797,84.427484,84.041656,84.279298,84.575410,78.892332,81.847419,82.207056,...,84.628600,84.782074,84.776176,84.852814,84.758480,84.752581,84.758480,84.776176,84.776176,84.793868,84.923495,84.723078,84.705372,84.770278,84.723078,84.687661,84.723078,84.681757,84.746681,84.876381,84.799764,84.593144,84.799764,84.799764,84.829240,84.829240,84.799764,84.829240,84.782074,84.728980,84.723078,84.870490,84.829240,84.799764,84.723078,84.782074,84.829240,84.799764,84.687661,84.687661
2,77.723870,0.000000,42.178193,47.413078,41.279535,41.460825,72.242647,60.074953,70.476947,48.733972,55.027266,64.923031,54.945427,63.229740,68.080834,41.844952,39.153544,38.444766,50.447993,46.904158,39.217343,64.350602,58.991525,46.173586,61.717096,43.749286,36.152455,58.694122,32.249031,43.611925,44.418465,47.801674,37.775654,38.561639,38.496753,37.416574,37.656341,34.307434,40.841156,39.433488,...,38.742741,38.301436,38.209946,38.405729,38.196859,38.209946,38.196859,38.209946,38.209946,38.457769,38.639358,38.327536,38.209946,38.691084,38.118237,38.691084,38.327536,38.366652,38.223030,38.535698,38.366652,38.431758,38.366652,38.366652,38.431758,38.431758,38.366652,38.431758,38.327536,38.262253,38.431758,38.522721,38.431758,38.366652,38.431758,38.327536,38.431758,38.366652,38.196859,38.196859
3,80.074965,42.178193,0.000000,52.124850,39.408121,34.727511,72.097157,62.577951,68.117545,44.609416,57.870545,69.526973,49.899900,58.043087,65.726707,34.380227,34.351128,31.256999,43.520110,39.357337,36.945906,70.823725,57.956881,45.617979,59.816386,38.509739,34.263683,65.513357,37.563280,37.000000,48.620983,42.497059,36.496575,30.561414,31.701735,31.480152,31.384710,40.620192,37.802116,35.355339,...,31.685959,31.048349,31.224990,31.368774,31.112698,31.032241,31.112698,31.224990,31.224990,31.112698,31.368774,30.983867,30.838288,31.432467,31.112698,31.432467,30.919250,30.903074,30.886890,31.240999,31.032241,31.622777,31.032241,31.032241,31.112698,31.112698,31.032241,31.112698,30.983867,31.096624,31.112698,31.224990,31.112698,31.032241,31.016125,30.983867,31.112698,31.032241,31.112698,30.822070
4,76.896034,47.413078,52.124850,0.000000,52.057660,54.927225,69.419018,58.008620,67.697858,56.753854,52.459508,60.074953,55.919585,65.726707,70.306472,53.768020,49.729267,53.084838,59.304300,55.623736,53.122500,61.212744,57.913729,54.662601,62.056426,50.239427,49.668904,59.203040,48.620983,53.516353,51.058790,52.507142,48.218254,52.754147,53.310412,52.402290,52.630789,48.425200,47.916594,50.507425,...,53.581713,53.188345,53.178943,53.301032,53.150729,53.141321,53.150729,53.178943,53.178943,53.282267,53.413481,53.188345,53.103672,53.075418,53.263496,53.450912,53.113087,53.065997,53.188345,53.338541,53.216539,53.188345,53.216539,53.216539,53.263496,53.263496,53.216539,53.263496,53.188345,53.141321,53.263496,53.329167,52.981129,53.028294,53.150729,53.188345,53.263496,53.216539,53.094256,53.037722
5,81.565924,41.279535,39.408121,52.057660,0.000000,36.646964,73.423430,61.016391,68.315445,47.738873,57.515215,67.786429,52.469038,60.926185,65.946948,36.124784,35.256205,32.557641,44.485953,42.918527,37.603191,70.092796,57.879185,50.398413,60.720672,38.704005,35.312887,63.804389,36.331804,37.629775,46.508064,43.874822,40.533936,31.953091,32.588341,31.144823,32.186954,36.810325,38.052595,33.985291,...,32.511536,31.859065,31.843367,32.046841,31.796226,31.780497

In [593]:
# top 5 similar movies to movie 1 
movie_movie_similarity[1].sort_values( )[1:6].index

Index([121, 405, 117, 151, 118], dtype='int64')

In [618]:
# finding most similar movies to movie i
for i in [1,2,5,10]:
    print('\n Top 5 similar movies to movie:',movies_df[movies_df['movie_id'] == i].movie_name.values[0],'are:')
    print(list(movies_df[movies_df['movie_id'].isin(list(movie_movie_similarity[i].sort_values( )[1:6].index))]['movie_name'].values))


 Top 5 similar movies to movie: Toy Story (1995) are:
['Rock, The (1996)', 'Twister (1996)', 'Independence Day (ID4) (1996)', 'Willy Wonka and the Chocolate Factory (1971)', 'Mission: Impossible (1996)']

 Top 5 similar movies to movie: GoldenEye (1995) are:
['Batman Forever (1995)', 'Under Siege (1992)', 'Cliffhanger (1993)', 'Demolition Man (1993)', 'Under Siege 2: Dark Territory (1995)']

 Top 5 similar movies to movie: Copycat (1995) are:
['Lord of Illusions (1995)', 'Congo (1995)', 'Drop Zone (1994)', 'Candyman: Farewell to the Flesh (1995)', 'Hideaway (1995)']

 Top 5 similar movies to movie: Richard III (1995) are:
['Kansas City (1996)', 'In the Bleak Midwinter (1995)', 'Hear My Song (1991)', 'Cronos (1992)', 'Twelfth Night (1996)']
